In [1]:
import numpy as np
import pandas as pd
import keras
import gc
import scipy.sparse as sparse
from scipy.sparse import coo_matrix
from keras.layers import Input, Embedding, Flatten, Dot, Add
from keras.models import Model

Using TensorFlow backend.


In [2]:
def OneHotEncoding_Cat(df, deletOrigingCol =True):
    df=df.copy()
    var = [i for i in df.columns.tolist() if i not in ['Rating','UserID','MovieID','movieID','TrainDataID']]
    for col in var:
        if (df[col].nunique()>2)|(df[col].dtypes == 'object'):
            df[col] = df[col].apply(lambda x : col + "_" + str(x))
            df = pd.concat([df , pd.get_dummies(df[col])],axis=1)
            if deletOrigingCol:
                del df[col]
            print('Transfrom ', col)
    return df    

In [3]:
train_df = pd.read_csv('train.csv')

In [4]:
userTrans_dict = {t: i for i, t in enumerate(train_df.UserID.unique())}
movieTrans_dict = {t: i for i, t in enumerate(train_df.MovieID.unique())}

In [5]:
train_df['UserID'] = train_df['UserID'].apply(lambda x : userTrans_dict[x])
train_df['MovieID'] = train_df['MovieID'].apply(lambda x : movieTrans_dict[x])

In [6]:
del train_df['TrainDataID']
gc.collect()
train_df.head()

,UserID,MovieID,Rating
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5


In [7]:
train = train_df.values
np.random.shuffle(train)

In [8]:
train_df.UserID.nunique(),train_df.MovieID.nunique()

(6040, 3688)

In [47]:
# Use Tensorflow tensor
user_input = Input(shape=(1,),name='User')
movie_input = Input(shape=(1,),name='Movie')

In [48]:
user_vec = Embedding(train_df.UserID.nunique(), 100, embeddings_initializer='random_normal',embeddings_regularizer='l2',name='Wu')(user_input)
user_bias = Embedding(train_df.UserID.nunique(), 1, embeddings_initializer='zeros',name='Bu')(user_input)
user_bias = Flatten()(user_bias)

In [49]:
movie_vec = Embedding(train_df.MovieID.nunique(), 100,embeddings_initializer='random_normal',embeddings_regularizer='l2',name='Wm')(movie_input)
movie_bias = Embedding(train_df.MovieID.nunique(), 1, embeddings_initializer='zeros',name='Bm')(movie_input)
movie_bias = Flatten()(movie_bias)

In [50]:
r_hat = Dot(axes=2)([user_vec, movie_vec])
r_hat = Flatten()(r_hat)
r_hat = Add()([r_hat, user_bias])
r_hat = Add()([r_hat, movie_bias])
model = Model([user_input, movie_input], r_hat)
model.compile(loss='mse',optimizer='sgd')

In [51]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
Wu (Embedding)                  (None, 1, 100)       604000      User[0][0]                       
__________________________________________________________________________________________________
Wm (Embedding)                  (None, 1, 100)       368800      Movie[0][0]                      
__________________________________________________________________________________________________
dot_5 (Dot

In [ ]:
output = model.fit([train[:, 0], train[:, 1]], train[:, 2], batch_size=64, epochs=100, verbose=2)

Epoch 1/100
 - 104s - loss: 5.4519
Epoch 2/100


In [46]:
uesr_embedding = model.layers[2].get_weights()[0]
movie_embedding = model.layers[3].get_weights()[0]

In [ ]:
movie = pd.read_csv('movies.csv',delimiter="::")

In [ ]:
movie['Genres'] = movie.Genres.apply(lambda x : x.split("|"))

In [ ]:
movie.head()

In [ ]:
movie_types = []
for i in movie['Genres']:
    movie_types += i
movie_types = list(set(movie_types))

In [ ]:
for movie_type in movie_types:
    movie[movie_type] = 0
    movie.loc[[(movie_type in i) for i in movie.loc[:,'Genres'].values],movie_type] = 1
del movie['Genres']
del movie['Title']
gc.collect()

In [ ]:
movie = movie.sort_values('movieID')

In [ ]:
user = pd.read_csv('users.csv',delimiter="::")

In [ ]:
user.head()

In [ ]:
train = OneHotEncoding_Cat(train,deletOrigingCol=False)

In [ ]:
train = (train.merge(user,on='UserID',how='left')
            .merge(movie,left_on='MovieID', right_on='movieID',how='left')
        )

In [ ]:
del train['TrainDataID']
gc.collect()

In [ ]:
train.head()

In [ ]:
user.head()

In [ ]:
user['Male'] = 0
user.loc[user.loc[:,'Gender']=='M','Male'] = 1
user['Age'] = user['Age'].apply(lambda x : int(x*0.1)*10)
del user['Gender']

In [ ]:
user = user.sort_values('UserID')

In [ ]:
user = OneHotEncoding_Cat(user)

In [ ]:
train.to_csv('TrainFinal.csv',index=False)

In [ ]:
out = keras.layers.Add()([input_a,input_b])
model = keras.models.Model([input_a,input_b],out)

In [ ]:
print(model.predict([a,b]))

In [ ]:
# map each repo and user to a unique numeric value
uid_job['uid'] = uid_job['uid'].astype("category")
uid_job['jobNo'] = uid_job['jobNo'].astype("category")
# create a sparse matrix of all the users/repos
sparse_matrix = coo_matrix((np.ones(uid_job.shape[0]),
                   (uid_job['jobNo'].cat.codes.copy(),
                    uid_job['uid'].cat.codes.copy())))